In [6]:
import xarray as xr
import fsspec
import s3fs
import os
import matplotlib.pyplot as plt
import dask
import datetime

In [ ]:
def convert_full_date_to_continous_day(year, month, day):
    return datetime.datetime(year, month, day).timetuple().tm_yday

def get_geo_data(year, month, day):
    fs = s3fs.S3FileSystem(anon=True)
    continous_day = convert_full_date_to_continous_day(year, month, day)
    file_location,var = fs.glob(f's3://noaa-goes17/ABI-L2-SSTF/'{str(year).zfill(4)}'/'{str(day).zfill(3)}'/*/*.nc'),'SST'
    
    #make a list of links to the file keys
    if len(file_location) < 1:
        raise Exception("No files found")

    file_ob = [fs.open(file) for file in file_location]        #open connection to files
    
    #open all the day's data
    ds = xr.open_mfdataset(file_ob,combine='nested',concat_dim='time') #note file is super messed up formatting
    
    #clean up coordinates which are a MESS in GOES
    #rename one of the coordinates that doesn't match a dim & should
    ds = ds.rename({'t':'time'})
    ds = ds.reset_coords()
    
    return ds